In [1]:
# Jannik's models are not SAE Lens, so use Baukit
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
# tiny model is in a different direction, add current one to path
import sys
from pathlib import Path
# Get the parent directory of current_dir
parent_dir = str(Path("/root/GroupedSAEs/notebooks/tiny_circuits.ipynb").parent.parent)
sys.path.append(parent_dir)
from tiny_model import TinyModel, tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TinyModel().to(device)

/root/GroupedSAEs/groups/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/root/GroupedSAEs/groups/lib/python3.11/site-packages/tiny_model/tokenization/tokenization.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serializati

In [2]:
from datasets import load_dataset, Dataset
from tqdm import tqdm
def download_dataset(dataset_name, tokenizer, max_length=256, num_datapoints=None):
    if(num_datapoints):
        split_text = f"train[:{num_datapoints}]"
    else:
        split_text = "train"
    dataset = load_dataset(dataset_name, split=split_text)
    print(dataset)
    total_failed_tokens = 0
    all_tokens = []
    for text in tqdm(dataset["text"]):
        try:
            tokens = [9996] + tokenizer.encode(text)[:max_length]
        except:
            total_failed_tokens += 1
            continue
        # only include if it's at least max_length
        if len(tokens) == max_length+1:
            all_tokens.append(tokens)
    print(f"Failed to tokenize {total_failed_tokens} tokens")
    # convert into a dataset class
    return  Dataset.from_dict({"input_ids": all_tokens})


dataset_name = "noanabeshima/TinyStoriesV2"
max_seq_length = 50
num_datapoints = 4000
# max_seq_length = 100
# num_datapoints = 500
print(f"Downloading {dataset_name}")
dataset = download_dataset(dataset_name, tokenizer=tokenizer, max_length=max_seq_length, num_datapoints=num_datapoints)
true_num_datapoints = len(dataset)
# added BOS
max_seq_length +=1
total_tokens = max_seq_length * true_num_datapoints
print(f"Number of datapoints w/ {max_seq_length} tokens: {true_num_datapoints}")
print(f"Total Tokens: {total_tokens / 1e6}M")


Dataset({
    features: ['text'],
    num_rows: 4000
})


100%|██████████| 4000/4000 [00:06<00:00, 655.61it/s]

Failed to tokenize 0 tokens
Number of datapoints w/ 51 tokens: 4000
Total Tokens: 0.204M


In [27]:
#set torch grad to zero globally
from torch.utils.data import DataLoader
from einops import rearrange
from tqdm import tqdm
from tiny_model import parse_mlp_tag, SparseMLP
N = 1
# activation_names = [f"A{N}", f"T{N}"]
# activation_names = [f"A{N}", f"A{N+1}"]
# activation_names = [f"T{N}", f"T{N+1}"]
activation_names = [f"Rm{N}_S-3_R1_P0", f"T{N}"]
sae_num = 0
activation_name = activation_names[sae_num]
# file, _, _, _ = parse_mlp_tag(activation_names[sae_num])
# sae = SparseMLP.from_pretrained(file).to(device)
# file, _, _, _ = parse_mlp_tag(activation_names[1])
# sae = SparseMLP.from_pretrained(file).to(device)

In [28]:
batch_size = 64
skip_first_n_pos = 0
# def get_dictionary_activations(model, dataset, cache_name, max_seq_length, autoencoder, batch_size=32):
num_features, d_model  = sae.encoder.weight.data.shape
datapoints = dataset.num_rows
all_dictionary_activations = torch.zeros((datapoints*max_seq_length, num_features))
token_list = torch.zeros((datapoints*max_seq_length), dtype=torch.int64)

FEAT_IDX=200

with torch.no_grad(), dataset.formatted_as("pt"):
    dl = DataLoader(dataset["input_ids"], batch_size=batch_size)
    for batch_ind, batch in enumerate(tqdm(dl)):
        batch = batch.to(model.device)
        token_list[batch_ind*batch_size*max_seq_length:(batch_ind+1)*batch_size*max_seq_length] = rearrange(batch, "b s -> (b s)")
        feature_acts = model[activation_name](batch)
        feature_acts[:, :skip_first_n_pos] = 0
        feature_acts = rearrange(feature_acts, "b s n -> (b s) n" )

        all_dictionary_activations[batch_ind*batch_size*max_seq_length:(batch_ind+1)*batch_size*max_seq_length] = feature_acts.cpu()

100%|██████████| 63/63 [00:37<00:00,  1.69it/s]


In [29]:
from interp_utils import *

num_feature_datapoints = 10
# features = [i for i in range(100)]
# features = [3484, 2094, 3085,  100,  708, 3220,  745, 1756, 2576,  320]
# features = [ 13, 320, 198, 311,  28, 143, 231, 238,  10,  95]
# features = [38, 6]
features = [i for i in range(40)]
# features = [6223, 27334, 28340, 9970, 16493]
# features = [14706, 32670, 10962, 3349, 16162]
# features = [7251]
ablate_context = False
# ablate_context = True
for feature in features:
    nz_ind_amount = all_dictionary_activations[:, feature].count_nonzero()
    print(f"feature: {feature}, non-zero activations: {nz_ind_amount}")
    if(nz_ind_amount == 0):
        continue
    uniform_indices = get_feature_indices(feature, all_dictionary_activations, k=num_feature_datapoints, setting="uniform")
    # uniform_indices = get_feature_indices(feature, all_dictionary_activations, k=num_feature_datapoints, setting="max")
    text_list, full_text, token_list, full_token_list, partial_activations, full_activations = get_feature_datapoints(uniform_indices, all_dictionary_activations[:, feature], tokenizer, max_seq_length, dataset)
    # logit_diffs = ablate_feature_direction(model, full_token_list, activation_name, max_seq_length, ae, feature = feature, batch_size=32, setting="sentences", model_type="causal")
    logit_diffs = None

    html = tokens_and_activations_to_html(full_token_list, full_activations, tokenizer, logit_diffs=logit_diffs)
    print(f"feature: {feature}")
    display(HTML(html))
    if(ablate_context):
        all_changed_activations = ablate_context_one_token_at_a_time(model, token_list, activation_name, ae, feature, max_ablation_length=10)
        # html = tokens_and_activations_to_html(full_token_list, full_activations, tokenizer, logit_diffs=logit_diffs)

        html = tokens_and_activations_to_html(token_list, all_changed_activations, tokenizer)
        print("Context_ablation\n===============")
        display(HTML(html))

feature: 0, non-zero activations: 462
feature: 0


feature: 1, non-zero activations: 171249
feature: 1


feature: 2, non-zero activations: 54631
feature: 2


feature: 3, non-zero activations: 24182
feature: 3


feature: 4, non-zero activations: 37880
feature: 4


...

feature: 35, non-zero activations: 22057
feature: 35


feature: 36, non-zero activations: 11863
feature: 36


feature: 37, non-zero activations: 43532
feature: 37


feature: 38, non-zero activations: 10921
feature: 38


feature: 39, non-zero activations: 3923
feature: 39
